In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../dependencies')

    
import pandas as pd
import numpy as np
#import xarray as xr
from dscore import mse, stl, quantiles, ilamb_score, scorecard_plot

ModuleNotFoundError: No module named 'matplotlib'

In [15]:
x = '../evaluation/example_data/01473000.prior.csv.zip'
#dir(x)

In [17]:
import zipfile
from glob import glob

data_path = '../evaluation/example_data/'

for f in glob(data_path + '*.zip'):
    with zipfile.ZipFile(f, 'r') as archive:
        #extract to notebook directory, or else set path=data_path
        archive.extractall(path='.') 

In [18]:
pd.read_csv('01473000.prior.csv')

,Unnamed: 0,modelled,measured,group,weight,standard_deviation
0,g_min_actet_mean_mon:10:5621,0.072515,0.026491,g_min_actet_mean_mon,2506.538170,0.000026
1,g_min_actet_mean_mon:10:5625,0.054129,0.035218,g_min_actet_mean_mon,2506.538170,0.000035
2,g_min_actet_mean_mon:10:5628,0.074993,0.027255,g_min_actet_mean_mon,2506.538170,0.000027
3,g_min_actet_mean_mon:10:5635,0.066819,0.036991,g_min_actet_mean_mon,2506.538170,0.000037
4,g_min_actet_mean_mon:10:5637,0.071988,0.034755,g_min_actet_mean_mon,2506.538170,0.000035
...,...,...,...,...,...,...
164115,streamflow_mon:2010_9:01472198,83.438700,15.563700,streamflow_mon,6.719053,1.556367
164116,streamflow_mon:2010_9:01472199,52.928200,10.906700,streamflow_mon,9.587999,1.090667
164117,streamflow_mon:2010_9:01472620,16.042900,36.813300,streamflow_mon,0.000000,3.681333
164118,streamflow_mon:2010_9:01472810,111.616000,51.806700,streamflow_mon,2.018526,5.180667


In [ ]:
def load_csv(filename, dataset_name):
    '''Helper function to load csv data
    '''
    data = pd.read_csv(filename, dtype={'HUC':str})
    data = data.melt(id_vars=["HUC"], 
                var_name="Date", 
                value_name=dataset_name)
    data["Date"] = pd.to_datetime(data["Date"], format="%Y_%m")
    data = data.set_index(['HUC', 'Date'])
    return data


In [ ]:
obs_file = 'data/waterwatch_monthly.csv'
nhm_file = 'data/nhm_c404bc_Streamflow_mm.csv'
nwm_file = 'data/wrfhydro_c404bc_Streamflow_mm.csv'

obs = load_csv(obs_file, 'obs')
nhm = load_csv(nhm_file, 'nhm')
nwm = load_csv(nwm_file, 'nwm')


data = pd.merge(obs, nhm, left_index=True, right_index=True)#.dropna(how='any')
data = pd.merge(data, nwm, left_index=True, right_index=True)
data = data.dropna(how='any')